In [4]:
json_out = '''
{
 "sub_process_list": [
{
 "start": "Capacity monitoring begins",
 "condition": "",
 "sequence_of_tasks": ["Monitor capacity utilization with BackEnd OPC", "Check 13-week and 26-week horizon"],
 "end": "Loading drops below threshold?"
},
{
 "start": "Loading drops below threshold?",
 "condition": "Yes",
 "sequence_of_tasks": ["Trigger assessment to determine cause of underloading"],
 "end": "Assess cause of underloading"
},
{
 "start": "Loading drops below threshold?",
 "condition": "No",
 "sequence_of_tasks": ["Continue monitoring capacity utilization"],
 "end": "Capacity monitoring continues"
},
{
 "start": "Assess cause of underloading",
 "condition": "Sub-bottleneck identified",
 "sequence_of_tasks": ["Investigate sub-bottlenecks or machine constraints", "Coordinate with BackEnd OPC to increase limiting capacity"],
 "end": "Coordinate with BackEnd OPC"
},
{
 "start": "Assess cause of underloading",
 "condition": "FrontEnd chip shortages",
 "sequence_of_tasks": ["Compare constrained plan against unconstrained plan", "Request SCP to increase FrontEnd chip supply", "Send supporting emails to pull chips into DieBank"],
 "end": "Increase FrontEnd chip supply"
},
{
 "start": "Assess cause of underloading",
 "condition": "Soft demand",
 "sequence_of_tasks": [
"Compare unconstrained plan to hardware-constrained demand",
"Review capacity line for non-production days and public holidays",
"Adjust bottleneck capacity for affected weeks",
"Pull manual data for constraint and safety stock plans",
"Calculate required capacity considering safety stocks and previous requests",
"Adjust required capacity for non-production days and public holidays",
"Ensure capacity utilization between 91% and 95% on average",
"Assess capacity line against ATV PP playbook",
"Initiate email or meeting with SCP to discuss capacity adjustments",
"Obtain SCM Head approval for adjustments or justifications for maintaining current capacity"
 ],
 "end": "Adjust capacity line"
},
{
 "start": "Coordinate with BackEnd OPC",
 "condition": "",
 "sequence_of_tasks": ["Update capacity", "Trigger DM refresh to reassess utilization and cut rates"],
 "end": "DM refresh triggered"
},
{
 "start": "Increase FrontEnd chip supply",
 "condition": "",
 "sequence_of_tasks": ["Receive updates on FrontEnd chip supply", "Reassess utilization post-refresh"],
 "end": "DM refresh triggered"
},
{
 "start": "Adjust capacity line",
 "condition": "",
 "sequence_of_tasks": ["Finalize capacity adjustments", "Trigger DM refresh to reassess utilization and cut rates"],
 "end": "DM refresh triggered"
},
{
 "start": "DM refresh triggered",
 "condition": "",
 "sequence_of_tasks": ["Validate that capacity line reflects all adjustments correctly"],
 "end": "Validate capacity updates"
},
{
 "start": "Validate capacity updates",
 "condition": "Updates align with inputs",
 "sequence_of_tasks": ["Ensure alignment with stakeholder inputs", "Continue monitoring capacity utilization"],
 "end": "Capacity monitoring continues"
},
{
 "start": "Validate capacity updates",
 "condition": "Updates do not align",
 "sequence_of_tasks": ["Reassess capacity plan with stakeholders", "Collaborate to develop improved capacity plan"],
 "end": "Reassess capacity plan"
},
{
 "start": "Reassess capacity plan",
 "condition": "",
 "sequence_of_tasks": ["Finalize reassessed capacity plan", "Continue monitoring capacity utilization"],
 "end": "Capacity monitoring continues"
}
 ]
}'''

In [7]:
import json

In [8]:
json_parsed = json.loads(json_out)

In [12]:
json_parsed

{'sub_process_list': [{'start': 'Capacity monitoring begins',
   'condition': '',
   'sequence_of_tasks': ['Monitor capacity utilization with BackEnd OPC',
    'Check 13-week and 26-week horizon'],
   'end': 'Loading drops below threshold?'},
  {'start': 'Loading drops below threshold?',
   'condition': 'Yes',
   'sequence_of_tasks': ['Trigger assessment to determine cause of underloading'],
   'end': 'Assess cause of underloading'},
  {'start': 'Loading drops below threshold?',
   'condition': 'No',
   'sequence_of_tasks': ['Continue monitoring capacity utilization'],
   'end': 'Capacity monitoring continues'},
  {'start': 'Assess cause of underloading',
   'condition': 'Sub-bottleneck identified',
   'sequence_of_tasks': ['Investigate sub-bottlenecks or machine constraints',
    'Coordinate with BackEnd OPC to increase limiting capacity'],
   'end': 'Coordinate with BackEnd OPC'},
  {'start': 'Assess cause of underloading',
   'condition': 'FrontEnd chip shortages',
   'sequence_of_t

In [ ]:
# this function takes as input a list of subprocesses: start:condition:sequence:end and trasnforms them into a connected list of elements
def parse_json_response_into_elements(parsed_json):
  id_counter = 1
  shared_id = {}
  unique_id = {}
  elements = []

  for subprocess in parsed_json["sub_process_list"]:
    start_element = subprocess["start"]
    if start_element not in shared_id.values():
      shared_id[f"{id_counter}"] = start_element
      id_counter += 1

    end_element = subprocess["end"]
    if end_element not in shared_id.values():
      shared_id[f"{id_counter}"] = end_element
      id_counter += 1

    #everything in between start and end is treated as unique item
    start_key = next((k for k, v in shared_id.items() if v == start_element), None)
    end_key = next((k for k, v in shared_id.items() if v == end_element), None)
    start_end_key = f"{start_key}_{end_key}"
    unique_id[start_end_key] = {}
    
    condition = subprocess["condition"]
    tasks = subprocess["sequence_of_tasks"]
    
    for task in tasks:
      unique_id[start_end_key][id_counter] = task
      id_counter += 1

    all_tasks = [start_element] + tasks + [end_element]

    for i in range(1, len(all_tasks)):
      print("length of elements", len(all_tasks))
      print("current at i", i)
      path_start = False
      path_end = False
      if i-1 == 0:
        path_start = True
      if i == len(all_tasks) -1:
        path_end = True
      
      print(path_start, path_end)

      from_elem = all_tasks[i-1]
      to_elem = all_tasks[i]
      outgoing_labels = []
      if path_start:
        # we fetch from shared keys
        start_key = next((k for k, v in shared_id.items() if v == from_elem), None)
        outgoing_labels = [condition]
      else:
        #we fetch from unique keys
        start_key = next((k for k, v in unique_id[start_end_key].items() if v == from_elem), None)

      if path_end:
        #we fetch from shared keys
        end_key = next((k for k, v in shared_id.items() if v == to_elem), None)

      else:
        #we fetch from unique keys
        end_key = next((k for k, v in unique_id[start_end_key].items() if v == to_elem), None)

      element = {"id": start_key, "type":"", "label":from_elem, "outgoing":[end_key], "outgoing_labels":outgoing_labels}
      elements.append(element)

    #add end elements without simultanous being start element as new end element
    for i in range(1, len(all_tasks)):
      if i == len(all_tasks) -1:
        path_end = True
      
      if path_end:
        path_end_key = next((k for k, v in shared_id.items() if v == all_tasks[i]), None)
        if all_tasks[i] not in [e["id"] for e in elements]:
          element = {"id": path_end_key, "type":"", "label":all_tasks[i], "outgoing":[], "outgoing_labels":[]}
          elements.append(element)
  return elements

def densen_connected_elements(elements):
    combined = {}

    for item in elements:
        key = item['id']
        if key in combined:
            # Combine the 'outgoing' and 'outgoing_labels' lists if id already exists
            combined[key]['outgoing'].extend(item['outgoing'])
            combined[key]['outgoing_labels'].extend(item['outgoing_labels'])
        else:
            combined[key] = item.copy()
    result_list = list(combined.values())
    return result_list

def identify_type(dense_elements):
    for e in dense_elements:
        outgoing_elements = len(e["outgoing"])
        if e["id"] == "1":
            e["type"] = "startEvent"
        elif outgoing_elements == 0:
            e["type"] = "endEvent"
        elif outgoing_elements == 1:
            e["type"] = "userTask"
        else:
            e["type"] = "exclusiveGateway"
    return dense_elements

elements = parse_json_response_into_elements(json_parsed)
dense_elements = densen_connected_elements(elements)
dense_elements = identify_type(dense_elements)